# Energy billing: Data generation

This notebook aggregates and generates the user energy consumption and/or production data used in the experimental verification of the Energy Billing project.

The data aggregated in this file, uses data generated by three other notebooks:
- [consumption](./consumption/consumption.ipynb), 
- [pv](./pv/pv.ipynb), and 
- [wind](./wind/wind.ipynb).

The code in this library was inspired by [the notebook in this repository](https://github.com/PeijieZ/Billing-Models-for-Electricity-Trading-Markets).

In [281]:
import pandas as pd
import random

In [282]:
# Settings
DAYS = 31
TIMESTEPS_PER_DAY = 24
TIMESTEPS = DAYS * TIMESTEPS_PER_DAY
HOUSEHOLDS = 100

DECIMAL_PRECISION = 4

## Load data

In [283]:
# Consumption data.
# 150 households, 24 intervals per day
CONSUMPTION = pd.read_json(f'consumption/out/consumption_{DAYS}_days.json')
CONSUMPTION

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0.0542,0.0812,0.1071,0.0391,0.0789,0.0950,0.0519,0.0609,0.3418,0.1033,...,0.3609,0.3091,0.0510,0.0620,0.0757,0.0537,0.1020,0.0503,0.1007,0.0874
1,0.0571,0.0813,0.1216,0.0392,0.0531,0.0487,0.0499,0.0532,0.1405,0.1084,...,0.1415,0.1477,0.0538,0.0505,0.0583,0.0555,0.1142,0.0517,0.0518,0.0519
2,0.0568,0.0686,0.1233,0.0378,0.1139,0.0472,0.0489,0.0685,0.1880,0.1197,...,0.1978,0.1758,0.0537,0.0677,0.1242,0.0581,0.1077,0.0512,0.0499,0.0439
3,0.1455,0.0768,0.1044,0.0377,0.0899,0.0468,0.1209,0.0458,0.2097,0.1024,...,0.2018,0.2118,0.1258,0.0496,0.0868,0.1323,0.1064,0.1444,0.0437,0.0489
4,0.0643,0.0812,0.1121,0.0404,0.1096,0.0728,0.0965,0.0700,0.2320,0.1033,...,0.2510,0.2442,0.1036,0.0736,0.1172,0.0706,0.1129,0.0626,0.0748,0.0760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,0.3874,0.4918,0.4450,0.4170,1.2683,0.6297,1.2006,1.0747,0.3930,4.6287,...,0.3368,0.3220,1.3014,0.9811,1.0456,0.4228,4.4558,0.4389,0.6108,0.7411
740,0.1626,0.3302,0.3597,0.5817,1.0791,0.0962,0.3495,0.7095,1.1060,4.1511,...,0.8709,1.2139,0.3595,0.7791,1.1198,0.1893,3.9073,0.1889,0.0943,0.1046
741,0.2380,0.3947,0.3788,0.5321,0.3883,0.1712,1.0451,1.1597,1.1474,0.6516,...,1.2406,1.2892,0.9608,1.2036,0.4117,0.2258,0.6550,0.2243,0.1754,0.1610
742,0.2529,0.3945,0.3436,0.3008,3.0879,0.2148,0.7100,1.1191,0.7212,0.3204,...,0.6657,0.6604,0.6750,1.2922,3.3265,0.2414,0.3830,0.2584,0.2676,0.2564


In [284]:
# PhotoVoltaic production data.
# 25 PV owners, measured in 24 intervals per day.
PV = pd.read_json(f'pv/out/solarpower_{DAYS}_days.json')
PV

,2.3W,3.6W,4.7W,21,20,19,18,17,16,15,...,9,8,7,6,5,4,3,2,1,0
2020-06-30 23:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-01 02:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-01 03:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-31 18:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-31 19:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-31 20:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-07-31 21:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [285]:
# WindTurbine production data.
# 10 WT owners, measured in 24 intervals per day.
WT = pd.read_json(f'wind/out/windpower_{DAYS}_days.json')
WT

,1kW,1.5kW,2kW,6,5,4,3,2,1,0
0,0.3847,0.5770,0.7694,0.3522,0.3817,0.5710,0.5629,0.4174,0.3470,0.8461
1,0.3974,0.5961,0.7948,0.3743,0.3722,0.5620,0.5862,0.3720,0.3622,0.8569
2,0.3935,0.5903,0.7870,0.4305,0.3701,0.5971,0.6312,0.3845,0.3870,0.8112
3,0.3766,0.5649,0.7532,0.3783,0.3825,0.5800,0.5162,0.3561,0.3804,0.7802
4,0.3714,0.5570,0.7427,0.4067,0.3771,0.5331,0.5130,0.3992,0.3519,0.7373
...,...,...,...,...,...,...,...,...,...,...
739,0.1784,0.2676,0.3567,0.1921,0.1867,0.2669,0.2438,0.1753,0.1954,0.3392
740,0.1898,0.2846,0.3795,0.1784,0.1970,0.2728,0.2687,0.1908,0.1929,0.3980
741,0.1965,0.2947,0.3930,0.2078,0.1882,0.3120,0.2952,0.2045,0.2060,0.3652
742,0.1967,0.2951,0.3935,0.2156,0.2055,0.2853,0.2793,0.2135,0.1816,0.3581


## Combine data

In [286]:
# Setup data structure

data = []
for _ in range(TIMESTEPS):
    ts_data = []

    for hh in range(HOUSEHOLDS):
        hh_stats = {
            'client ID': hh,
            'feed in tarif': 0.05
        }
        ts_data.append(hh_stats)
    
    data.append(ts_data)
print(len(data))

744


In [287]:
# Aggregate consumption and production data
for ts_idx, ts in enumerate(data):

    pv_ts_data = PV.iloc[ts_idx].T
    wt_ts_data = WT.iloc[ts_idx].T

    # Aggregate data for this timestep
    for hh, stats in enumerate(ts):

        # Consumption data
        consumption = CONSUMPTION[hh]
        consumption = consumption[ts_idx]
        stats['consumption'] = consumption

        # WT data: add for 0 - 9
        if hh < 10:
            wt = wt_ts_data.iloc[hh]
        else:
            wt = 0
        stats['wind'] = wt

        # PV data: add for 5 - 29
        if 5 <= hh < 30:
            pv = pv_ts_data.iloc[hh - 5]    
        else:
            pv = 0
        stats['pv'] = pv

        # Supply
        stats['supply'] = supply = wt + pv

        # Consumption/production profile
        cp_profile = supply - consumption
        stats['cp profile'] = round(cp_profile, DECIMAL_PRECISION)

        ts_data[hh] = stats

df = pd.DataFrame(data[16]).T
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
client ID,0.0000,1.0000,2.0000,3.0000,4.0000,5.0000,6.0000,7.0000,8.0000,9.0000,...,90.0000,91.0000,92.0000,93.0000,94.0000,95.0000,96.0000,97.0000,98.0000,99.0000
feed in tarif,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,...,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500,0.0500
consumption,0.0564,0.2740,0.1176,0.0284,0.0530,0.0976,0.3307,0.3805,0.1628,0.1813,...,0.1746,0.0481,0.3773,0.0309,0.0538,0.1992,0.1704,0.3616,0.0534,0.3114
wind,0.4965,0.7448,0.9930,0.5207,0.4524,0.7698,0.7626,0.5256,0.4871,0.9739,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
pv,0.0000,0.0000,0.0000,0.0000,0.0000,0.6737,1.0545,1.3767,1.4621,1.2549,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
supply,0.4965,0.7448,0.9930,0.5207,0.4524,1.4435,1.8171,1.9023,1.9492,2.2288,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
cp profile,0.4401,0.4708,0.8754,0.4923,0.3994,1.3459,1.4864,1.5218,1.7864,2.0475,...,-0.1746,-0.0481,-0.3773,-0.0309,-0.0538,-0.1992,-0.1704,-0.3616,-0.0534,-0.3114


# Add further statistics

In [288]:
# Generate 'predictions' of a user's consumption
predictions_set =  15 * [("perfect",(0.00, 0.00))] \
                 + 90 * [("good",   (0.01, 0.10))] \
                 + 30 * [("ok",     (0.11, 0.20))] \
                 + 15 * [("bad",    (0.21, 1.00))]
predictions_set = predictions_set[:HOUSEHOLDS]
assert len(predictions_set) == HOUSEHOLDS
random.shuffle(predictions_set)


for ts_idx, ts in enumerate(data):
    for stats, prediction_settings in zip(ts, predictions_set):
        category, interval = prediction_settings
        
        # predict based on
        # cp_profile = stats['cp profile']
        supply = stats['supply']
        consumption = stats['consumption']

        # Determine errors
        supply_error = supply * random.uniform(*interval) * random.choice([-1, 1])
        consumption_error = consumption * random.uniform(*interval) * random.choice([-1, 1])

        pred_supply = round(supply + supply_error, DECIMAL_PRECISION)
        pred_consumption = round(consumption + consumption_error, DECIMAL_PRECISION)

        stats['prediction algorithm'] = category
        stats['consumption prediction'] = abs(pred_consumption)
        stats['supply prediction'] = abs(pred_supply)

df = pd.DataFrame(data[13]).T
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
client ID,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
feed in tarif,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
consumption,0.0894,0.8032,0.1242,0.0379,0.1881,0.0825,0.1469,0.1339,0.1873,0.2862,...,0.2971,0.1729,0.1417,0.0399,0.1848,0.3063,0.268,0.141,0.0967,0.1499
wind,0.3616,0.5424,0.7232,0.3593,0.3883,0.4998,0.5884,0.3456,0.3287,0.6747,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pv,0.0,0.0,0.0,0.0,0.0,0.7634,1.1949,1.56,1.6485,1.5539,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
supply,0.3616,0.5424,0.7232,0.3593,0.3883,1.2632,1.7833,1.9056,1.9772,2.2286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cp profile,0.2722,-0.2608,0.599,0.3214,0.2002,1.1807,1.6364,1.7717,1.7899,1.9424,...,-0.2971,-0.1729,-0.1417,-0.0399,-0.1848,-0.3063,-0.268,-0.141,-0.0967,-0.1499
prediction algorithm,good,good,good,good,good,good,good,good,good,good,...,good,good,perfect,good,good,good,good,good,perfect,good
consumption prediction,0.0885,0.8773,0.1315,0.0366,0.1824,0.0788,0.1519,0.1363,0.1939,0.3142,...,0.2768,0.1759,0.1417,0.0404,0.181,0.2928,0.2537,0.1339,0.0967,0.1459
supply prediction,0.3458,0.5145,0.7568,0.3304,0.4133,1.1747,1.9601,1.8253,1.9378,2.1488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [289]:
# Add supplier information
# Generate random supplier information
supplier_info_set =  45 * [("supplier1", 0.200)] \
                   + 45 * [("supplier2", 0.205)] \
                   + 30 * [("supplier3", 0.210)] \
                   + 15 * [("supplier4", 0.215)] \
                   + 15 * [("supplier3", 0.220)]
random.shuffle(supplier_info_set)
supplier_info_set = supplier_info_set[:HOUSEHOLDS]
assert len(supplier_info_set) == HOUSEHOLDS

# Assign a supplier to each consumer
for ts in data:
    for stats, supplier_info in zip(ts, supplier_info_set):
        supplier, rate = supplier_info
        stats['supplier'] = supplier
        stats['retail price'] = rate

df = pd.DataFrame(data[16]).T
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
client ID,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
feed in tarif,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
consumption,0.0564,0.274,0.1176,0.0284,0.053,0.0976,0.3307,0.3805,0.1628,0.1813,...,0.1746,0.0481,0.3773,0.0309,0.0538,0.1992,0.1704,0.3616,0.0534,0.3114
wind,0.4965,0.7448,0.993,0.5207,0.4524,0.7698,0.7626,0.5256,0.4871,0.9739,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pv,0.0,0.0,0.0,0.0,0.0,0.6737,1.0545,1.3767,1.4621,1.2549,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
supply,0.4965,0.7448,0.993,0.5207,0.4524,1.4435,1.8171,1.9023,1.9492,2.2288,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cp profile,0.4401,0.4708,0.8754,0.4923,0.3994,1.3459,1.4864,1.5218,1.7864,2.0475,...,-0.1746,-0.0481,-0.3773,-0.0309,-0.0538,-0.1992,-0.1704,-0.3616,-0.0534,-0.3114
prediction algorithm,good,good,good,good,good,good,good,good,good,good,...,good,good,perfect,good,good,good,good,good,perfect,good
consumption prediction,0.0585,0.2962,0.1075,0.027,0.0521,0.1014,0.3077,0.3622,0.1593,0.1978,...,0.192,0.0516,0.3773,0.0289,0.0496,0.2158,0.1678,0.3368,0.0534,0.3227
supply prediction,0.4697,0.7949,0.8955,0.5396,0.4232,1.551,1.6536,2.0225,2.1396,2.4067,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Run auction

In [290]:
BUYER_PRICE_INTERVAL = 0.05, 0.19
SELLER_PRICE_INTERVAL = 0.06, 0.20
TRADING_PRICE = 0.11
FEED_IN_TARIF = 0.05

def get_random_price(interval):
    return round(random.uniform(*interval), 2)

for ts_idx, ts in enumerate(data):

    # Determine role for this round
    buyers, sellers = [], []
    for hh_stats in ts:

        pred_supply = hh_stats['supply prediction']
        pred_consumption = hh_stats['consumption prediction']
        pred_effective = pred_supply - pred_consumption
        
        if pred_effective >= 0:
            role = "seller"
            sellers.append(hh_stats)
            hh_stats['consumption promise'] = 0
            hh_stats['supply promise'] = hh_stats['supply prediction'] - hh_stats['consumption prediction']
        else:
            role = "buyer"
            buyers.append(hh_stats)
            hh_stats['consumption promise'] = hh_stats['consumption prediction'] - hh_stats['supply prediction'] 
            hh_stats['supply promise'] = 0

        hh_stats['actual consumption'] = max(0, hh_stats['consumption'] - hh_stats['supply'])
        hh_stats['actual supply'] = max(0, hh_stats['supply'] - hh_stats['consumption'])

        hh_stats['role'] = role

    # Give buyers a random trading price
    for buyer in buyers:
        price = get_random_price(BUYER_PRICE_INTERVAL)
        buyer['trading price'] = price

        # Accept if greater than trading price
        if price >= TRADING_PRICE:
            buyer['bid'] = "accepted"
        else:
            buyer['bid'] = "rejected"

    # Give sellers a random trading price
    for seller in sellers:
        price = get_random_price(SELLER_PRICE_INTERVAL)
        seller['trading price'] = price

        # Accept if smaller than trading price
        if price <= TRADING_PRICE:
            seller['bid'] = "accepted"
        else:
            seller['bid'] = "rejected"

for ts_idx, ts in enumerate(data):    
    # Record number of consumers and prosumers
    accepted_sellers = list(filter(lambda x: x['bid'] == "accepted" and x['role'] == "seller", ts))
    nr_accepted_sellers = len(accepted_sellers)
    accepted_buyers = list(filter(lambda x: x['bid'] == "accepted" and x['role'] == "buyer", ts))
    nr_accepted_buyers = len(accepted_buyers)

    for hh_stats in ts:
        hh_stats['total acc. prosumers'] = max(nr_accepted_sellers, 1) ## prevent division by zero problems; should not impact results
        hh_stats['total acc. consumers'] = max(nr_accepted_buyers, 1) ## prevent division by zero problems; should not impact results


df = pd.DataFrame(data[14]).T
df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
client ID,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
feed in tarif,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
consumption,0.1119,0.6026,0.114,0.0391,0.055,0.1764,0.1956,0.2923,0.1405,0.3348,...,0.3491,0.0552,0.3034,0.0396,0.0497,0.3067,0.3067,0.1823,0.1108,0.2069
wind,0.3914,0.5871,0.7827,0.4066,0.3565,0.543,0.5801,0.3898,0.426,0.7708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pv,0.0,0.0,0.0,0.0,0.0,0.6769,1.0595,1.3832,1.3094,1.5065,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
supply,0.3914,0.5871,0.7827,0.4066,0.3565,1.2199,1.6396,1.773,1.7354,2.2773,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cp profile,0.2795,-0.0155,0.6687,0.3675,0.3015,1.0435,1.444,1.4807,1.5949,1.9425,...,-0.3491,-0.0552,-0.3034,-0.0396,-0.0497,-0.3067,-0.3067,-0.1823,-0.1108,-0.2069
prediction algorithm,good,good,good,good,good,good,good,good,good,good,...,good,good,perfect,good,good,good,good,good,perfect,good
consumption prediction,0.1092,0.6508,0.1085,0.0382,0.0508,0.1711,0.1767,0.3192,0.1476,0.3145,...,0.3786,0.0607,0.3034,0.0364,0.0468,0.2824,0.3009,0.1859,0.1108,0.198
supply prediction,0.4095,0.5545,0.8111,0.4025,0.3808,1.1184,1.6769,1.6011,1.8777,2.4326,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Total deviation

In [291]:
# Get deviation of a user
def get_deviation(stats):
    if stats['role'] == "buyer":
        consumption_prediction = stats['consumption prediction'] - stats['supply prediction']
        effective_consumption = stats['consumption'] - stats['supply']
        deviation = effective_consumption - consumption_prediction

    elif stats['role'] == "seller":
        supply_prediction = stats['supply prediction'] - stats['consumption prediction']
        effective_supply = stats['supply'] - stats['consumption']
        deviation = effective_supply - supply_prediction

    return deviation

# Compute total deviation among accepted buyers and sellers
def get_total_deviation(data):

    # Get buyer deviation
    buyers = filter(lambda x: x['role'] == "buyer", data)
    accepted_buyers = list(filter(lambda x: x['bid'] == "accepted", buyers))
    
    buyer_deviation = 0
    for buyer in accepted_buyers:
        buyer_deviation += get_deviation(buyer)

    sellers = filter(lambda x: x['role'] == "seller", data)
    accepted_sellers = list(filter(lambda x: x['bid'] == "accepted", sellers))

    seller_deviation = 0
    for seller in accepted_sellers:
        seller_deviation += get_deviation(seller)

    return seller_deviation - buyer_deviation


# Compute bill for everybody
for ts_idx, ts in enumerate(data):
    total_deviation = get_total_deviation(ts)

    for hh_stats in ts:
        hh_stats['total deviation'] = total_deviation
        hh_stats['deviation'] = get_deviation(hh_stats)
        hh_stats['accepted'] = 1 if hh_stats['bid'] == "accepted" else 0
        # hh_stats['accepted'] = 0

In [292]:
# Overwrite trading price to be 11 cents
for ts in data:
    for hh_stats in ts:
        hh_stats['trading price'] = TRADING_PRICE

In [293]:
# Compute bill and reward
for ts_idx, ts in enumerate(data):
    for hh_stats in ts:
        total_dev = hh_stats['total deviation']

        if hh_stats['accepted'] == 1:
            # P2P trading
            bill = hh_stats['actual consumption'] * TRADING_PRICE
            reward = hh_stats['actual supply'] * TRADING_PRICE

            if total_dev < 0 and hh_stats['deviation'] > 0 and hh_stats['role'] == "buyer":
                bill += total_dev / hh_stats['total acc. consumers'] * (hh_stats['retail price'] - TRADING_PRICE)

            if total_dev > 0 and hh_stats['deviation'] > 0 and hh_stats['role'] == "seller":
                reward += total_dev / hh_stats['total acc. prosumers'] * (FEED_IN_TARIF - TRADING_PRICE)

        else:
            # No p2p trading
            bill = hh_stats['actual consumption'] * hh_stats['retail price']
            reward = hh_stats['actual supply'] * FEED_IN_TARIF

        hh_stats['bill'] = bill
        hh_stats['reward'] = reward

## Export data

In [294]:
# Convert to user-oriented dataslices
user_slices = []
for u_idx in range(HOUSEHOLDS):
    user_slice = []
    for ts in data:
        user_slice.append(ts[u_idx])
    user_slices.append(user_slice)

user_frames = []
for slice in user_slices:
    df = pd.DataFrame(slice).T
    user_frames.append(df)
    
user_frames[1]

,0,1,2,3,4,5,6,7,8,9,...,734,735,736,737,738,739,740,741,742,743
client ID,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
feed in tarif,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
consumption,0.0812,0.0813,0.0686,0.0768,0.0812,0.0757,0.0704,1.0703,0.0877,0.0702,...,0.0662,0.4084,0.0828,0.095,0.696,0.4918,0.3302,0.3947,0.1039,0.1039
wind,0.577,0.5961,0.5903,0.5649,0.557,0.5783,0.7384,0.8074,0.8069,0.7569,...,0.4439,0.4195,0.2992,0.2262,0.2587,0.2676,0.2846,0.2947,0.3995,0.3995
pv,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
supply,0.577,0.5961,0.5903,0.5649,0.557,0.5783,0.7384,0.8074,0.8069,0.7569,...,0.4439,0.4195,0.2992,0.2262,0.2587,0.2676,0.2846,0.2947,0.3995,0.3995
cp profile,0.4958,0.5148,0.5217,0.4881,0.4758,0.5026,0.668,-0.2629,0.7192,0.6867,...,0.3777,0.0111,0.2164,0.1312,-0.4373,-0.2242,-0.0456,-0.1,0.2956,0.2956
prediction algorithm,good,good,good,good,good,good,good,good,good,good,...,good,good,good,good,good,good,good,good,good,good
consumption prediction,0.0775,0.0785,0.0676,0.0827,0.0801,0.0721,0.0655,1.1618,0.0908,0.0732,...,0.0643,0.4208,0.0783,0.1036,0.6537,0.4636,0.3369,0.3695,0.1111,0.1111
supply prediction,0.5567,0.6334,0.5653,0.5385,0.6085,0.6002,0.8117,0.7922,0.8494,0.7673,...,0.4266,0.4484,0.3205,0.2446,0.2757,0.261,0.294,0.2988,0.427,0.427


In [295]:
# strip unused values

user_frames = [
    df.drop([
        'consumption',
        'supply',
        'client ID',
        'feed in tarif',
        'wind',
        'pv',
        'cp profile',
        'prediction algorithm',
        'supplier',
        'role',
        'trading price',
        'total acc. consumers',
        'total acc. prosumers',
        'total deviation',
        'consumption prediction',
        'supply prediction',
        'bid'
    ])
    for df in user_frames
]

user_frames[1]

,0,1,2,3,4,5,6,7,8,9,...,734,735,736,737,738,739,740,741,742,743
retail price,0.205,0.205,0.205,0.205,0.205,0.205,0.205,0.205,0.205,0.205,...,0.205,0.205,0.205,0.205,0.205,0.205,0.205,0.205,0.205,0.205
consumption promise,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3696,0.0,0.0,...,0.0,0.0,0.0,0.0,0.378,0.2026,0.0429,0.0707,0.0,0.0
supply promise,0.4792,0.5549,0.4977,0.4558,0.5284,0.5281,0.7462,0.0,0.7586,0.6941,...,0.3623,0.0276,0.2422,0.141,0.0,0.0,0.0,0.0,0.3159,0.3159
actual consumption,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2629,0.0,0.0,...,0.0,0.0,0.0,0.0,0.4373,0.2242,0.0456,0.1,0.0,0.0
actual supply,0.4958,0.5148,0.5217,0.4881,0.4758,0.5026,0.668,0.0,0.7192,0.6867,...,0.3777,0.0111,0.2164,0.1312,0.0,0.0,0.0,0.0,0.2956,0.2956
deviation,0.0166,-0.0401,0.024,0.0323,-0.0526,-0.0255,-0.0782,-0.1067,-0.0394,-0.0074,...,0.0154,-0.0165,-0.0258,-0.0098,0.0593,0.0216,0.0027,0.0293,-0.0203,-0.0203
accepted,0,0,0,0,0,1,0,1,0,1,...,0,1,0,0,0,1,1,0,0,0
bill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028919,0.0,0.0,...,0.0,0.0,0.0,0.0,0.089647,0.022654,0.005016,0.0205,0.0,0.0
reward,0.02479,0.02574,0.026085,0.024405,0.02379,0.055286,0.0334,0.0,0.03596,0.075537,...,0.018885,0.001221,0.01082,0.00656,0.0,0.0,0.0,0.0,0.01478,0.01478


In [296]:
# Export individual user data
from pathlib import Path
data_dir = Path(f"./data/{TIMESTEPS}_ts_{HOUSEHOLDS}_clients/")
data_dir.mkdir(exist_ok=True)
for idx, user in enumerate(user_frames):
    user = user.T.iloc[:TIMESTEPS].T    
    user.to_csv(data_dir / f"user_{idx}.csv")

# Export bill context data
context = pd.DataFrame(user_slices[0]).T
context = context.loc[[
    'feed in tarif',
    'trading price',
    'total acc. consumers',
    'total acc. prosumers',
    'total deviation']
].T.iloc[:TIMESTEPS].T
context.to_csv(data_dir / "context.csv")

## Future work

### Improved auction

In [297]:
# Introduce the buyer / seller statistic

BUYER_PRICE_LOW, BUYER_PRICE_HIGH = 0.05, 0.19
BUYER_VOL_GRADIENT = 2

def get_buyer_bid(pred_consumption_vol) -> float:
    """
    Scale price linearly with consumption,
    increasing as volume increases.
    """
    scale = min(pred_consumption_vol / BUYER_VOL_GRADIENT, 1)
    bid = (1-scale) * BUYER_PRICE_LOW + scale * BUYER_PRICE_HIGH
    return round(bid, 2)


SELLER_PRICE_LOW, SELLER_PRICE_HIGH = 0.06, 0.20
SELLER_VOL_GRADIENT = 3.5

def get_seller_offer(pred_production_vol) -> float:
    """
    Scale price linearly with production,
    decreasing as volume increases.
    """
    scale = min(pred_production_vol / SELLER_VOL_GRADIENT, 1)
    offer = scale * SELLER_PRICE_LOW + (1-scale) * SELLER_PRICE_HIGH
    return round(offer, 2)


for ts_idx, ts in enumerate(data):

    # Determine role for this round
    buyers, sellers = [], []
    for hh_stats in ts:
        prediction = hh_stats['prediction']
        
        if prediction >= 0:
            role = "seller"
            sellers.append(hh_stats)
        else:
            role = "buyer"
            buyers.append(hh_stats)

        hh_stats['role'] = role

    # Determine price offer
    offers, bids = [], []
    for hh_id, hh_stats in enumerate(ts):
        prediction = abs(hh_stats['prediction'])

        if hh_stats in buyers:
            pred_cons_vol = abs(prediction)
            price = get_buyer_bid(pred_cons_vol)
            bids.append((hh_id, price, pred_cons_vol))
        elif hh_stats in sellers:
            pred_prod_vol = abs(prediction)
            price = get_seller_offer(pred_prod_vol)
            offers.append((hh_id, price, pred_prod_vol))
        else:
            raise Exception("woops, something went wrong here.")
        hh_stats['offer'] = price
    
    # Run the auction
    assert len(offers) + len(bids) == HOUSEHOLDS

    # Sort offers and bids: lowest to highest price AND largest to smallest volume
    offers.sort(key=lambda x: (x[1], -x[2]))
    bids.sort(key=lambda x: (x[1], -x[2]))

    while len(offers) > 0 and len(bids) > 0:
        # Try to match cheapest offer with highest bid
        highest_offer = offers.pop()
        highest_bid = bids.pop()
        seller_id, offer, seller_vol = highest_offer
        buyer_id, bid, buyer_vol = highest_bid

        if offer > bid:
            # Reject seller
            ts[seller_id]['bid'] = "rejected"

            # Return bidder to pool
            bids.append(highest_bid)

            continue

        if seller_vol >= buyer_vol:
            # Buy the buyer out
            ts[buyer_id]['bid'] = "accepted"

            # Return the remaining offer to pool
            remaining = seller_vol - buyer_vol
            if remaining > 0:
                rem_offer = (seller_id, offer, remaining)
                offers.append(rem_offer)

        if buyer_vol >= seller_vol:
            # Buy the seller out
            ts[seller_id]['bid'] = "accepted"

            # Return remaining bid to pool
            remaining = buyer_vol - seller_vol
            if remaining > 0:
                rem_bid = (buyer_id, bid, remaining)
                bids.append(rem_bid)
        
    # Set remaining bids/offers to rejected
    for seller_id, _, _ in offers:
        ts[seller_id]['bid'] = "rejected"
    for buyer_id, _, _ in bids:
        ts[buyer_id]['bid'] = "rejected"

df = pd.DataFrame(data[142]).T
df

KeyError: 'prediction'

### Compute rewards

In [ ]:
# Get deviation of a user
def get_deviation(stats):

    if stats['role'] == "buyer":
        pred_cons = abs(stats['prediction'])
        act_cons = abs(stats['cp profile'])
        deviation = act_cons - pred_cons

    elif stats['role'] == "seller":
        pred_prod = stats['prediction']
        act_prod = stats['cp profile']
        deviation = act_prod - pred_prod

    return deviation

# Compute total deviation among accepted buyers and sellers
def get_total_deviation(data):

    # Get buyer deviation
    buyers = filter(lambda x: x['role'] == "buyer", data)
    accepted_buyers = list(filter(lambda x: x['bid'] == "accepted", buyers))
    
    buyer_deviation = 0
    for buyer in accepted_buyers:
        buyer_deviation += get_deviation(buyer)

    sellers = filter(lambda x: x['role'] == "seller", data)
    accepted_sellers = list(filter(lambda x: x['bid'] == "accepted", sellers))

    seller_deviation = 0
    for seller in accepted_sellers:
        seller_deviation += get_deviation(seller)

    return seller_deviation - buyer_deviation


# Compute bill for everybody
for ts_idx, ts in enumerate(data):
    total_deviation = get_total_deviation(ts)

    for hh_id, stats in enumerate(ts):


        if total_deviation > 0:
            indiv_deviation = get_deviation(stats)

            supply = stats['cp profile']
            price = stats['offer']

            if indiv_deviation < 0:
                # you get paid for your stuff
                reward = supply * price


            if indiv_deviation > 0:
                # Assumes everybody has same feed-in tarif!
                # which is not the case in the dataset!

                feed_in_tarif = stats['retail price']

                

                reward = (supply - indiv_deviation) * price
                reward = indiv_deviation * feed_in_tarif

                reward = ? + (total_deviation / total_positive_prosumers (?)) * (feed_in_tarif - trading_price)
                                                                          
            

        elif total_deviation == 0:
        
        elif total_deviation < 0:

        
        

    if total_deviation < 0:


            deviation = get_deviation(stats)


        

    
